In [ ]:
import torch
import pandas as pd
import torch.utils
from torch import nn

In [ ]:
#Loading house prices training and test dataset
hptrain = pd.read_csv("/Users/bartoszgawin/Desktop/ML/Practise/PyTorch_Basic/Datasets/hptrain.csv",index_col=0)
hptest = pd.read_csv("Datasets/hptest.csv",index_col=0)

In [ ]:
#Transforming data into tensor data format from pandas' dataframe
X_ols_train = torch.tensor(hptrain[['LotArea', 'OverallQual', 'OverallCond', 'MasVnrArea', 'TotalBsmtSF']].values)
Y_ols_train = torch.tensor(hptrain[['SalePrice']].values)
X_ols_train = X_ols_train.float()
Y_ols_train = Y_ols_train.float()

#Splitting data into iterable batches
olstensor = torch.utils.data.TensorDataset(X_ols_train,Y_ols_train)
olsdataloader = torch.utils.data.DataLoader(olstensor, batch_size=20, shuffle=True)

In [ ]:
#Specifying the paramenters of our model
olsnet = nn.Sequential(nn.Linear(X_ols_train.size()[1], Y_ols_train.size()[1]))
olsnet[0].weight.data.normal_(0, 0.1)
olsnet[0].bias.data.normal_(0, 0.1)
olsloss = nn.MSELoss(reduction='sum')
olstrainer = torch.optim.SGD(olsnet.parameters(),lr=0.03)

In [ ]:
olsnet[0].weight.data

In [ ]:
olsnet[0].weight.data.normal_(0, 0.1)
olsnet[0].bias.data.normal_(0, 0.1)

num_epochs = 5
for epoch in range(num_epochs):
    print(f'{epoch+1}th epoch')
    X, Y = next(iter(olsdataloader))
    X = X.float()
    Y = Y.float()
    l = olsloss(olsnet(X),Y)
    print(f'Loss:{l.item()}')
    olstrainer.zero_grad()
    l.backward()
    olstrainer.step()
    print('Coefficients: ',olsnet[0].weight.data)



